<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/ENEM%204bit%203-shot%20GPTj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U rank_bm25
!pip install -q -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 19.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 16.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# ---------------------------------------------------------- #
# This code is from https://github.com/piresramon/gpt-4-enem #
# ---------------------------------------------------------- #

import json
import torch
import random 
import collections

import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('max_colwidth', 400)

from tqdm.auto import tqdm

import rank_bm25

from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score
from datasets import Dataset

if torch.cuda.is_available(): 
    device = 'cuda'  
else: 
    device ='cpu'

MANUAL_SEED = 2711
rnd = random.Random()
rnd.seed(MANUAL_SEED)
def deterministic(rep=True, manual_seed=MANUAL_SEED):
    if rep:
        torch.manual_seed(manual_seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(manual_seed)
            torch.cuda.manual_seed_all(manual_seed)
        torch.backends.cudnn.enabled = False 
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        print(f'Experimento deterministico, seed: {manual_seed}')
        if device == 'cuda':
            print(f'Existe {torch.cuda.device_count()} GPU\
            {torch.cuda.get_device_name(0)} disponível.')
    else:
        print('Experimento randomico')
deterministic()    

Experimento deterministico, seed: 2711
Existe 1 GPU            NVIDIA A100-SXM4-40GB disponível.


# Get Enem json and create enem_data 

In [3]:
def ignore_question(doc):
    filters = {
        'IU': False,
        # 'MR': False,  # uncomment to filter out MR
        # 'CE': False,  # uncomment to filter out CE
        'ML': False,
    }
    for k,v in filters.items():
        if doc[k] != v:
            return True
    return False

def _process_doc_cot(doc):
    def format_example(doc, choices):
        prompt = "Contexto: " + doc["context"] + "\n"
        prompt += "Enunciado: " + doc["question"] + "\nAlternativas:\n"
        for choice, option in zip(choices, doc["options"]):
            prompt += f"{choice.upper()}. {option}\n"
        
        prompt += "Explicação: " + doc.get("explanation", "")
        return prompt.strip()
    choices = ['a', 'b', 'c', 'd', 'e']
    return {
        "query": format_example(doc, choices),
        "choices": doc["options"],
        "gold": choices.index(doc["label"]),
        "id": doc["id"],
        "exam": doc["exam"],
    }    

dataset = collections.defaultdict(list)
        
data_path = "/content/drive/MyDrive/LLMs/ENEM/ENEMdataset/2022.json"
with open(data_path) as f:
    documents = json.load(f)

documents = list(filter(lambda doc: not ignore_question(doc), documents))
dataset['test'] = list(map(_process_doc_cot, documents))

enem_data = {ix:doc  for ix, doc in enumerate(dataset['test'])}

print(f"tamanho dataset: {len(enem_data)}")

enem_data[0]    

tamanho dataset: 118


{'query': 'Contexto: A conquista da medalha de prata por Rayssa Leal, no skate street nos Jogos Olímpicos, é exemplo da representatividade feminina no esporte, avalia a âncora do jornal da rede de televisão da CNN. A apresentadora, que também anda de skate, celebrou a vitória da brasileira, que entrou para a história como a atleta mais nova a subir num pódio defendendo o Brasil. “Essa representatividade do esporte nos Jogos faz pensarmos que não temos que ficar nos encaixando em nenhum lugar. Posso gostar de passar notícia e, mesmo assim, gostar de skate, subir montanha, mergulhar, andar de bike, fazer yoga. Temos que parar de ficar enquadrando as pessoas dentro de regras. A gente vive num padrão no qual a menina ganha boneca, mas por que também não fazer um esporte de aventura? Por que o homem pode se machucar, cair de joelhos, e a menina tem que estar sempre lindinha dentro de um padrão? Acabamos limitando os talentos das pessoas”, afirmou a jornalista, sobre a prática do skate por m

# Create Prompts
- #### Dynamic Prompt (fewshot sampled)
- #### Dynamic Similar Prompt (bm25 rank fewshot)

In [4]:
def dynamic_similar_prompt(doc_id, data, topk=3):
    key_predict = [k for k, v in enem_data.items() if v['id'].startswith(doc_id)][0]
    query_list = [v['query'] for k,v in data.items()]
    tokenized_corpus = [doc.split(" ") for doc in query_list]
    bm25 = rank_bm25.BM25Plus(tokenized_corpus)

    query = data[key_predict]['query']
    tokenized_query = query.split(" ")
    
    doc_scores = bm25.get_scores(tokenized_query)
    
    # [1:] remove query with itself ---> topk+1
    top_similar_idxs = list(doc_scores.argsort()[-(topk+1):][::-1])[1:]
    if key_predict in top_similar_idxs: # k in topk_sim ---> must be false
        print('Error BM25 prompt similar retrieval')
    
    return top_similar_idxs


def dynamyc_fewshot_examples(num_fewshot, doc_id, sim_prompt=True, data=enem_data):
    if sim_prompt:
        topk_sim = dynamic_similar_prompt(doc_id, data, topk=num_fewshot)
        fewshot_ex = [enem_data[k] for k in topk_sim]

    else:
        # filter the sample in current predict from data
        all_possible_fewshot_keys = [k for k, v in data.items() if not v['id'].startswith(doc_id)]
        fewshot_keys = rnd.sample(all_possible_fewshot_keys, num_fewshot)
        fewshot_ex = [enem_data[k] for k in fewshot_keys]

    return fewshot_ex


def fewshot_context(doc, num_fewshot, similar_prompt=True, enem_data=enem_data):
    if num_fewshot == 0 and fewshotex is None:
        labeled_examples = ""
    else:
        fewshotex = dynamyc_fewshot_examples(
            num_fewshot=num_fewshot, doc_id=doc['id'], sim_prompt=similar_prompt, data=enem_data
        )
        
        labeled_examples = "Dado o exemplo, forneça uma resposta após ##Explicação:## não faça nenhuma explicação adicional" + "\n"
        for i, doc_ex in enumerate(fewshotex):
            labeled_examples += f'Questão {i+1}:\n'
            labeled_examples += doc_ex['query'] + " " + ['A.', 'B.', 'C.', 'D.', 'E.'][doc_ex['gold']].upper()
            labeled_examples += '\n##\n'
        labeled_examples += f'Questão {len(fewshotex) + 1}:\n'

    example = doc['query']
    
    return labeled_examples + example

# -------------------------------------------------------------------------------------- #
# number of fewshots, must be > 0

# create inputs ---> list with fewshot samples + question to be answer 
one_shot, two_shot, three_shot = [],[],[]
for k,doc in enem_data.items():
    one_shot.append(fewshot_context(doc=doc,   num_fewshot=1, similar_prompt=False, enem_data=enem_data))
    two_shot.append(fewshot_context(doc=doc,   num_fewshot=2, similar_prompt=False, enem_data=enem_data))
    three_shot.append(fewshot_context(doc=doc, num_fewshot=3, similar_prompt=False, enem_data=enem_data))

In [5]:
one_shot[0]

'Dado o exemplo, forneça uma resposta após ##Explicação:## não faça nenhuma explicação adicional\nQuestão 1:\nContexto: Macrocefalia urbana pode ser entendida como a massiva concentração das atividades econômicas em algumas metrópoles que propicia o desencadeamento de processos descompassados: redirecionamento e convergência de fluxos migratórios, déficit no número de empregos, ocupação desordenada de determinadas regiões da cidade e estigmatização de estratos sociais, que comprometem substancialmente a segurança pública urbana. SANTOS, M. O espaço dividido: os dois circuitos da economia urbana dos países subdesenvolvidos. São Paulo: Edusp, 2004.\nEnunciado: O processo de concentração espacial apresentado foi estimulado por qual fator geográfico?\nAlternativas:\nA. Limitação da área ocupada.\nB. Êxodo da população do campo.\nC. Ampliação do risco habitacional. D\nD. Deficiência do transporte alternativo.\nE. Crescimento da taxa de fecundidade.\nExplicação: B.\n##\nQuestão 2:\nContexto:

In [6]:
def get_results(trues, preds, chatgpt=False):
    dataframe = pd.DataFrame({'true': trues, 'pred':preds})

    dataframe['pred'] = dataframe.pred.apply(lambda x: x[-1:])
    
    map_label = {0:"A", 1:"B", 2:"C", 3:"D", 4:"E"}
    dataframe['true'] = dataframe.true.apply(lambda x: map_label[x])
    
    acc = accuracy_score(dataframe['true'], dataframe['pred'])
    
    return dataframe, acc

In [7]:
model_gpt = "EleutherAI/gpt-j-6B"
tokenizer_gpt = AutoTokenizer.from_pretrained(model_gpt)

def get_input_lenghts(dataset, tokenizer):
    num_words = [len(x.split()) for x in dataset]
    num_words = torch.tensor(num_words, dtype=torch.float)
    tokens_lengths = [len(tokenizer.encode(seq)) for seq in dataset]
    tokens_lengths = torch.tensor(tokens_lengths, dtype=torch.float)
    
    return int(torch.ceil(num_words.mean()).item()), tokens_lengths

# -------------------------------------------------------------------- #
num_words, gpt2_lengths = get_input_lenghts(three_shot, tokenizer_gpt)
min_gpt2 = torch.ceil(gpt2_lengths.min()).item()
max_gpt2 = torch.ceil(gpt2_lengths.max()).item()
mean_gpt2 = torch.ceil(gpt2_lengths.mean()).item()
# -------------------------------------------------------------------- #
print('\n\n')
print('--'*33)
print(f' Prompt 3-shot com média de {num_words} palavras em  ENEM2022 dataset')
print('--'*33)
print(f'GPT2 tem média de {int(mean_gpt2)} tokens em 3-shot')

Token indices sequence length is longer than the specified maximum sequence length for this model (2317 > 2048). Running this sequence through the model will result in indexing errors





------------------------------------------------------------------
 Prompt 3-shot com média de 750 palavras em  ENEM2022 dataset
------------------------------------------------------------------
GPT2 tem média de 1804 tokens em 3-shot


# Eval 4-bit GPTj

In [8]:
model = AutoModelForCausalLM.from_pretrained(model_gpt, load_in_4bit=True, device_map="auto")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [ ]:
deterministic()    

trues, preds = [], []
loop = tqdm(one_shot, leave=True)

for ix, batch in enumerate(loop):

    # to make sure the same example to predict has the correct label 
    # find the point where the question to be evaluate starts
    point = one_shot[ix].find('Questão 2:\n')
    
    # "11+point" ---> is the start of the query-text, so this is a naive approach
    if enem_data[ix]['query'] == one_shot[ix][11+point:]:
        inputs = tokenizer_gpt.encode(one_shot[ix], return_tensors="pt", max_length=2048).to(device)
        outputs = model.generate(inputs)
        
        preds.append(tokenizer_gpt.decode(outputs[0]))
        trues.append(enem_data[ix]['gold'])
    else:
        # if the query-text is different from inputs to be evaluated 
        # we print for further investigation
        print(point, ix)

In [12]:
preds[0]

'Dado o exemplo, forneça uma resposta após ##Explicação:## não faça nenhuma explicação adicional\nQuestão 1:\nContexto: Macrocefalia urbana pode ser entendida como a massiva concentração das atividades econômicas em algumas metrópoles que propicia o desencadeamento de processos descompassados: redirecionamento e convergência de fluxos migratórios, déficit no número de empregos, ocupação desordenada de determinadas regiões da cidade e estigmatização de estratos sociais, que comprometem substancialmente a segurança pública urbana. SANTOS, M. O espaço dividido: os dois circuitos da economia urbana dos países subdesenvolvidos. São Paulo: Edusp, 2004.\nEnunciado: O processo de concentração espacial apresentado foi estimulado por qual fator geográfico?\nAlternativas:\nA. Limitação da área ocupada.\nB. Êxodo da população do campo.\nC. Ampliação do risco habitacional. D\nD. Deficiência do transporte alternativo.\nE. Crescimento da taxa de fecundidade.\nExplicação: B.\n##\nQuestão 2:\nContexto:

In [10]:
dataframe = pd.DataFrame({'true': trues, 'pred':preds})
dataframe.pred.apply(lambda x: x[-1:])

0       A
1       A
2       A
3       A
4       A
5       E
6       E
7       A
8       A
9       A
10      A
11      E
12      A
13     \n
14      A
15      A
16      A
17      A
18      A
19      E
20      A
21      A
22      A
23      E
24      A
25      A
26      A
27      E
28      A
29      A
30      A
31      A
32      E
33      A
34      A
35      E
36      A
37      A
38      E
39      A
40      A
41      A
42      A
43      E
44      A
45      E
46      E
47      A
48      E
49      A
50      E
51      A
52      A
53      A
54      A
55      A
56      A
57      A
58      A
59      A
60      A
61      E
62      A
63      A
64      A
65      A
66      A
67      E
68      A
69      A
70      E
71      A
72      D
73      E
74      A
75      A
76      E
77      A
78      A
79      A
80      E
81      D
82      A
83      A
84      A
85      A
86      A
87      A
88      A
89      B
90      E
91      A
92      A
93      A
94      A
95      E
96      A
97      E
98      A
99      E


In [14]:
gptj, acc = get_results(trues, preds)

print(f'ACC of 1-shot GPT0J on ENEM-2022: {acc:.3}')

ACC of 1-shot GPT0J on ENEM-2022: 0.229
